<a href="https://colab.research.google.com/github/ptripathi18/EZ/blob/main/prerna_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy.sql import text
import xml.etree.ElementTree as ET
import glob
from sqlalchemy import create_engine

# Specify the path to the TMX files
tmx_ = glob.glob('/content/ar-en.tmx')

# Define the XML namespace for the 'xml' prefix
xml_namespace = {'xml': 'http://www.w3.org/XML/1998/namespace'}

# Initialize lists to store English and Arabic texts
english = []
arabic = []

# Parse the TMX files and extract translation pairs
for i in tmx_:
    tree = ET.parse(i)
    root = tree.getroot()

    for tu in root.findall('.//body/tu'):
        english_text = tu.find(".//tuv[@xml:lang='en']/seg", namespaces=xml_namespace).text
        arabic_text = tu.find(".//tuv[@xml:lang='ar']/seg", namespaces=xml_namespace).text
        english.append(english_text)
        arabic.append(arabic_text)

# Create a DataFrame
df = pd.DataFrame({'English': english, 'Arabic': arabic})

In [ ]:
df.head()

,English,Arabic
0,RESOLUTION 55/100,القرار 55/100
1,"Adopted at the 81st plenary meeting, on 4 Dece...",اتخذ في الجلسة العامة 81، المعقودة في 4 كانون ...
2,"In favour: Afghanistan, Algeria, Angola, Antig...",المؤيدون: الاتحاد الروسي، إثيوبيا، الأرجنتين، ...
3,"Abstentions: Albania, Andorra, Australia, Aust...",الممتنعون: أذربيجان، إسبانيا، أستراليا، إستوني...
4,55/100. Respect for the right to universal fre...,55/100- احترام حق الجميع في حرية السفر والأهمي...


In [ ]:
# Create a database connection
co = sqlalchemy.create_engine('mssql+pyodbc://RATNESHWAR\MSSQLSERVER01/tmx_trans?trusted_connection=yes&driver=ODBC Driver 17 for SQL Server')

In [ ]:
query = ("""
CREATE TABLE  tmx(
    English NVARCHAR(max),
    Arabic NVARCHAR(max) COLLATE Arabic_CI_AS
);
""")
with co.connect() as connection:
    connection.execute(text(query))

# Write the DataFrame to the database
df.to_sql("tmx", con=co, if_exists="replace", index=False, dtype={'English': sqlalchemy.NVARCHAR(), 'Arabic': sqlalchemy.NVARCHAR()})